In [10]:
using 
    Plots,
    StatPlots,
    RecipesBase,
    Colors, 
    Images,
    DataFrames,
    DataFramesMeta,
    ConjugatePriors,
    Particles

using Revise
Revise.track("modeling.jl")
using DotLearning
include("experiments.jl")
include("plots.jl")

using JLD2
@load "data/dots2014.jld2"
@load "prior_empirical.jld2"
@load "results/run3-2018-07-18T12:40:07.059.jld2" results expts

2-element Array{Symbol,1}:
 :results
 :expts  

# Judgements are made in context

# Spatial memory task

In [14]:
recall1 = @where(recall, :subjid1 .== first(:subjid1))
p = plot([], [], xlim=(-1,1), ylim=(-1,1), seriestype=:scatter, legend=false)
@gif for (x,y) in zip(recall1[:x], recall1[:y])
    push!(p, x,y)
end

INFO: Saved animation to /home/dave/work/dots-location-memory-pernille/tmp.gif


Plots.AnimatedGif("/home/dave/work/dots-location-memory-pernille/tmp.gif")